In [1]:
#basics
import pandas as pd 
import numpy as np
import os
import sys

#parser
import urllib
from bs4 import BeautifulSoup

#viz
import matplotlib.pyplot as plt

#NLP
import re
import string
import unicodedata
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem.snowball import SnowballStemmer
# import spacy
from nltk import pos_tag 
# Lemmatize using WordNet’s built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet.
from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import sent_tokenize,word_tokenize
# Tweet tokenizer does not split at apostophes which is what we want
from nltk.tokenize import TweetTokenizer 

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download("maxent_treebank_pos_tagger")
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mjjackey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mjjackey\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mjjackey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\mjjackey\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mjjackey\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
eng_stopwords = set(stopwords.words("english")) #stopwords
# warnings.filterwarnings("ignore")
tokenizer=TweetTokenizer() #tokenizer
lem = WordNetLemmatizer()  #stem,Lemmatize using WordNet’s built-in morphy function. Returns the input word unchanged if it cannot be found in WordNet
stemmer = SnowballStemmer("english")

In [42]:
# 有效代码
cwd=os.getcwd()
fd_str=os.path.join(cwd,"output3")
print(fd_str)

D:\Code\Jupyter Notebook\COMP6237_Data_Mining\output3


In [43]:
# 有效代码
def read_file(file_path):
#     with open(file_path, 'rt',encoding='latin-1') as f:
    with open(file_path, 'rt',encoding='utf-8') as f:
        data = f.read()
    return data

In [44]:
# 有效代码
def write_file(file_path,text):
    with open(file_path,'w',encoding='utf-8') as f:
        f.write(text)

In [45]:
# 有效代码
doc_list=[]
filenames=[]
for name in os.listdir(fd_str):
    if name.endswith('.txt') and os.path.isfile(os.path.join(fd_str, name)):
        filenames.append(name)
        file_path=os.path.join(fd_str, name)
        text=read_file(file_path) 
        doc_list.append(text)

In [46]:
print(len(doc_list),filenames[1])

24 gap_2X5KAAAAYAAJ.txt


In [47]:
print(len(doc_list[1]))

430085


In [10]:
def retain_ascii(doc_text):
    doc_text = unicodedata.normalize('NFD', str(doc_text))
    doc_text=doc_text.encode('ascii', 'ignore').decode('ascii')
    return doc_text

In [11]:
'''
doc_text_list is list，只保留空格和字母，适合没有分词前对text做pre-clean
'''
def retain_letter_space(doc_text_list):
    doc_text=''.join(e for e in doc_text_list if e.isalpha() or e.isspace())
    return doc_text

In [12]:
'''
doc_text_list is list
'''
def retain_letter_space1(doc_text_list):
    doc_text=' '.join(e for e in doc_text_list if e.isalpha() or e.isspace())
    return doc_text

In [13]:
'''
doc_text_list is list，保留了单个字母（缩写含.）的人名的given name
'''
def retain_letter2(doc_text_list):
    doc_text=' '.join(e for e in doc_text_list if len(e)==2 and e[0].isalpha() and e[1]=='.' or e.isalpha() and len(e)>=2) 
#     doc_text=''
#     for e in doc_text_list:
# #         print(e)
#         if len(e)==2 and e[0].isalpha() or e.isalpha() or e.isspace():
# #             print('clean:'+e)
#             doc_text=doc_text+' '+e
# #             print('doc_text:'+doc_text)
    return doc_text

In [32]:
'''
doc_text_list is list，去除单个字母（缩写含.）的人名的given name
'''
def retain_letter3(doc_text_list):
    doc_text=' '.join(e for e in doc_text_list if e.isalpha() and len(e)>=2) 
    return doc_text

In [14]:
def clean_romanumber(s):
#     remap={
#         ord('i'):'',
#         str('ii'):'',
#        ord('iii'):'',
#         ord('iv'):'',
#         ord('v'):'',
#         ord('vi'):'',
#         ord('vii'):'',
#         ord('viii'):'',
#         ord('ix'):'',
#         ord('x'):'',
#     }

#     s=s.replace('viii','')
#     s=s.replace('vii','')
#     s=s.replace('vi','')
#     s=s.replace('v','')
#     s=s.replace('iv','')
#     s=s.replace('iii','')
#     s=s.replace('ix','')
#     s=s.replace('x','')
#     s=s.replace('ii','')
#     s=s.replace('i','')

#     regex=r'^i$|^ii$|^iii$|^iv$|^v$|^vi$|^vii$|^viii$|^ix$|^x$'
    regex=r'i|ii|iii|iv|v|vi|vii|viii|ix|x'
    s=re.sub(regex,'',s)
    return s

In [15]:
'''
不能去除特殊符号，慢
'''
def clean_punctuation(doc_text):
    remove_punct_map = dict.fromkeys(map(ord, string.punctuation))
    return doc_text.translate(remove_punct_map)

In [16]:
'''
可以去除特殊符号，快
'''
def clean_punctuation2(doc_text):
    tbl=dict.fromkeys(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))
    return doc_text.translate(tbl)

In [17]:
'''
标注词形
'''
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

In [18]:
def preclean(doc_text):
    #remove 'OCR Output'
    doc_text=doc_text.replace('OCR Output','')
    #convert to lower case
    doc_text=doc_text.lower()
    #just retain the letter and the space char,简写的.会去掉,比如人名的given name
#     doc_text=retain_letter_space(doc_text)
    #remove roma number(3-10)
#     doc_text=clean_romanumber(doc_text)
    return doc_text

In [19]:
def preclean2(doc_text):
    #remove 'OCR Output'
    doc_text=doc_text.replace('OCR Output','')
    #convert to lower case
    doc_text=doc_text.lower()
    #just retain the letter and the space char,简写的.会去掉,比如人名的given name
    doc_text=retain_letter_space(doc_text)
    #remove roma number(3-10)
#     doc_text=clean_romanumber(doc_text)
    return doc_text

In [20]:
'''
按空格和标点符号分词；
分词后去除单个字母；
使用WordNetLemmatizer，默认词性是verb
'''
def clean1(text):
    #1.Pre-clean
    text=preclean(text)
    #2.toknenize
    text_list=tokenizer.tokenize(text)  #按空格和标点符号分
    #remove one letter
    text_list = [w for w in text_list if len(w)>=2]
    #3.remove stopwords
    text_list = [w for w in text_list if not w in eng_stopwords] 
    #4.lemmatize,stem
    text_list = [lem.lemmatize(word, "v") for word in text_list] 
    text = retain_letter_space1(text_list)
    return text 

In [21]:
'''
先按句分，再按词分；
分词后去除单个字母；
使用WordNetLemmatizer，默认词性是verb
'''
def clean2(text):
    #1.Pre-clean
    text=preclean(text)
    #2.toknenize
#     text=tokenizer.tokenize(text)  #按空格和标点符号分
    text_list=[word_tokenize(t) for t in sent_tokenize(text)] #先按句分，再按词分，还是会把人名的given name简写的.去掉（依然把它当句号）
    print(text_list)
    #remove one letter
    text = [w for sent_list in text_list for w in sent_list if len(w)>=2]
    print(text)
    #3.remove stopwords
    text = [w for w in text if not w in eng_stopwords] 
    #4.lemmatize,stem
    text = [lem.lemmatize(word, "v") for word in text] 
    clean_text=" ".join(text)
    return clean_text

In [22]:
'''
使用SnowballStemmer；
保留了单个字母的人名的given name
'''
def clean3(text):
    #1.Pre-clean
    text=preclean(text)
    #2.toknenize
    text_list=[word_tokenize(t) for t in sent_tokenize(text)] #先按句分，再按词分，还是会把人名的given name简写的.去掉
#     print(text_list)
    
    #remove one letter
#     text = [w for sent_list in text_list for w in sent_list if len(w)>=2]
#     print(text)

    #3.remove stopwords
    text_list = [w for sent_list in text_list for w in sent_list if not w in eng_stopwords]
#     print(text)
    #4.stem
    text_list = [stemmer.stem(word) for word in text_list] 
#     print(text)
    text = retain_letter2(text_list)
    return text

In [23]:
'''
使用WordNetLemmatizer，默认词性是verb；
保留了单个字母的人名的given name（含.）
'''
def clean4(text):
    #1.Pre-clean
    text=preclean(text)
    #2.toknenize
    text_list=[word_tokenize(t) for t in sent_tokenize(text)] #先按句分，再按词分
#     print(text_list)
    
    #remove one letter
#     text = [w for sent_list in text_list for w in sent_list if len(w)>=2]
#     print(text)

    #3.remove stopwords
    text_list = [w for sent_list in text_list for w in sent_list if not w in eng_stopwords]
#     print(text)
    #4.stem
    text_list = [lem.lemmatize(word, "v") for word in text_list] #list
#     print(text)
    text = retain_letter2(text_list)
#     clean_text=" ".join(text)
    return text

In [24]:
'''
使用WordNetLemmatizer, 使用pos_tag标注词性；
保留了单个字母的人名的given name（含.）
'''
def clean5(text):
    #1.Pre-clean
    text=preclean(text)
    #2.toknenize
    text_list=[word_tokenize(t) for t in sent_tokenize(text)] #先按句分，再按词分，返回二维列表
#     print("Toknenize:",text_list)
 
    #3.stem
    res_list=[]
    for t in text_list:
        for word,pos in pos_tag(t):
            wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
            res_list.append(lem.lemmatize(word, pos=wordnet_pos))
#     print("Stem:",res_list)
    
    #remove one letter
#     text = [w for sent_list in text_list for w in sent_list if len(w)>=2]
#     print(text)

    #3.remove stopwords
    text_list = [w for w in res_list if not w in eng_stopwords]
#     print("Remove Stopwords:",text_list)

    text = retain_letter2(text_list)
    return text

In [25]:
'''
使用WordNetLemmatizer, 使用pos_tag标注词性；
去掉了单个字母的人名的given name的简写（含.）
'''
def clean6(text):
    #1.Pre-clean
    text=preclean2(text)
    #2.toknenize
    text_list=[word_tokenize(t) for t in sent_tokenize(text)] #先按句分，再按词分
#     print("Toknenize:",text_list)

    #remove one letter
    text_list = [w for sent_list in text_list for w in sent_list if len(w)>=2]
#     print("remove one letter:",text_list)
 
    #3.stem
    res_list=[]
    for word,pos in pos_tag(text_list):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res_list.append(lem.lemmatize(word, pos=wordnet_pos))
#     print("Stem:",res_list)
    
    #3.remove stopwords
    text_list = [w for w in res_list if not w in eng_stopwords]
#     print("Remove Stopwords:",text_list)

    text=" ".join(text_list)
    return text

In [38]:
'''
使用WordNetLemmatizer, 使用pos_tag标注词性；
去掉了单个字母的人名的given name的简写（含.）
'''
def clean7(text):
    #1.Pre-clean
    text=preclean(text)
    #2.toknenize
    text_list=[word_tokenize(t) for t in sent_tokenize(text)] #先按句分，再按词分
#     print("Toknenize:",text_list)

    #remove one letter
    text_list = [w for sent_list in text_list for w in sent_list if len(w)>=2]
#     print("remove one letter:",text_list)
 
    #3.stem
    res_list=[]
    for word,pos in pos_tag(text_list):
        wordnet_pos = get_wordnet_pos(pos) or wordnet.NOUN
        res_list.append(lem.lemmatize(word, pos=wordnet_pos))
#     print("Stem:",res_list)
    
    #3.remove stopwords
    text_list = [w for w in res_list if not w in eng_stopwords]
#     print("Remove Stopwords:",text_list)

    text = retain_letter3(text_list)
    return text

In [33]:
a = 'C. B. a Special [$#!^...]#~ \r \n characters. [$#!^ like jumping jumped dixit the spaces of D f eye 888323 dictionaries said leaves tortures torturing right Rights'
retain_letter_space(a)

'C B a Special  \r \n characters  like jumping jumped dixit the spaces of D f eye  dictionaries said leaves tortures torturing right Rights'

In [211]:
retain_ascii(a)

'C. B. a Special [$#!^...]#~ characters [$#!^ like jumping jumped dixit the spaces of D f eye 888323 dictionaries said leaves'

In [212]:
clean_punctuation2(a)

'C B a Special $^~ characters $^ like jumping jumped dixit the spaces of D f eye 888323 dictionaries said leaves'

In [213]:
clean_punctuation(a)

'C B a Special  characters  like jumping jumped dixit the spaces of D f eye 888323 dictionaries said leaves'

In [214]:
b='i ii iii iv a v vi vii viii ix x b'
clean_romanumber(b)

'    a       b'

In [215]:
preclean(a)

'c. b. a special [$#!^...]#~ characters [$#!^ like jumping jumped dixit the spaces of d f eye 888323 dictionaries said leaves'

In [216]:
clean1(a)

'special character like jump jump dixit space eye dictionaries say leave'

In [217]:
clean2(a)

[['c.', 'b.', 'a', 'special', '[', '$', '#', '!', '^', '...', ']', '#', '~', 'characters', '[', '$', '#', '!', '^', 'like', 'jumping', 'jumped', 'dixit', 'the', 'spaces', 'of', 'd', 'f', 'eye', '888323', 'dictionaries', 'said', 'leaves']]
['c.', 'b.', 'special', '...', 'characters', 'like', 'jumping', 'jumped', 'dixit', 'the', 'spaces', 'of', 'eye', '888323', 'dictionaries', 'said', 'leaves']


'c. b. special ... character like jump jump dixit space eye 888323 dictionaries say leave'

In [218]:
clean3(a)

'c. b. special charact like jump jump dixit space eye dictionari said leav'

In [219]:
clean4(a)

'c. b. special character like jump jump dixit space eye dictionaries say leave'

In [220]:
clean5(a)

'c. b. special character like jump jump dixit space eye dictionary say leaf'

In [30]:
clean6(a)

'special character like jump jump dixit space eye dictionary say leave torture torture right right'

In [37]:
clean7(a)

Toknenize: [['c.', 'b.', 'a', 'special', '[', '$', '#', '!', '^', '...', ']', '#', '~', 'characters', '.'], ['[', '$', '#', '!', '^', 'like', 'jumping', 'jumped', 'dixit', 'the', 'spaces', 'of', 'd', 'f', 'eye', '888323', 'dictionaries', 'said', 'leaves', 'tortures', 'torturing', 'right', 'rights']]


'special character like jump jump dixit space eye dictionary say leave torture torture right right'

In [48]:
# clean6(doc_list[1])

In [49]:
# clean7(doc_list[1])

In [50]:
fd_store_dir=os.path.join(cwd,"clean_output3")
if(not os.path.exists(fd_store_dir)):
        os.mkdir(fd_store_dir)
for i in range(len(doc_list)):
# for i in range(2):
    print("i:"+str(i))
   
    name=filenames[i]
    name2=name.split(".")[0]
    file_store_path=os.path.join(fd_store_dir,name2+".txt")

    text=clean7(doc_list[i])
    write_file(file_store_path,text)
    
    


i:0
i:1
i:2
i:3
i:4
i:5
i:6
i:7
i:8
i:9
i:10
i:11
i:12
i:13
i:14
i:15
i:16
i:17
i:18
i:19
i:20
i:21
i:22
i:23
